In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
#https://api.tidesandcurrents.noaa.gov/api/prod/

url = 'https://api.tidesandcurrents.noaa.gov/api/prod/'
parameters = 'datagetter?date=latest&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=json'

r = requests.get(url + parameters)

r.status_code

200

In [3]:
r.json()

{'metadata': {'id': '9410170',
  'name': 'San Diego',
  'lat': '32.7156',
  'lon': '-117.1767'},
 'data': [{'t': '2025-01-12 21:53', 'v': '7.927'}]}

In [4]:
parameters = 'datagetter?begin_date=20250110&end_date=20250112&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=csv'
r = requests.get(url + parameters)

r.status_code

200

In [5]:
csv_file = open('test.csv', 'wb')
csv_file.write(r.content)
csv_file.close()

In [ ]:
paramters = 'datagetter?date=latest&station=9410170&product=one_minute_water_level&datum=STND&time_zone=lst&units=english&format=json'
